# Alpha Vantage Stock Data Analysis

This notebook demonstrates comprehensive stock market data analysis using Alpha Vantage API with LangChain integration for investment research.

## Features

- Real-time and historical stock price data
- Technical indicators and market analysis
- Company fundamentals and financial metrics
- LangChain integration for intelligent insights
- Risk assessment and trend analysis
- Investment recommendation framework

## Prerequisites

- Alpha Vantage API key (free from https://www.alphavantage.co/support/#api-key)
- Required Python packages: requests, pandas, langchain, python-dotenv


# Install required packages


In [16]:
%pip install requests pandas langchain langchain-text-splitters python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
# Import required libraries
import requests
import pandas as pd
import json
from datetime import datetime, timedelta
import os
from typing import Dict, List, Optional, Any, Tuple
import time
import math

# LangChain imports
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate

# Environment variables
from dotenv import load_dotenv
load_dotenv()

print("Libraries imported successfully!")
print(f"Analysis started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Libraries imported successfully!
Analysis started at: 2025-09-20 22:57:54


## Configuration and Setup

Configure API credentials and analysis parameters for stock market analysis.


In [18]:
# Configuration
ALPHA_VANTAGE_API_KEY = os.getenv('ALPHA_VANTAGE_API_KEY')
if not ALPHA_VANTAGE_API_KEY:
    print("Warning: ALPHA_VANTAGE_API_KEY not found in environment variables.")
    print("Please set your Alpha Vantage API key in the .env file.")
    print("Get your free API key from: https://www.alphavantage.co/support/#api-key")
else:
    print("Alpha Vantage API key loaded successfully!")

# Alpha Vantage API configuration
ALPHA_VANTAGE_BASE_URL = "https://www.alphavantage.co/query"
REQUEST_TIMEOUT = 30
RATE_LIMIT_DELAY = 12  # Free tier: 5 calls per minute

# Analysis parameters
TEST_TICKER = "AAPL"  # Default ticker for analysis
ANALYSIS_PERIOD = "3month"  # Time period for analysis

print(f"Configured for ticker: {TEST_TICKER}")
print(f"Analysis period: {ANALYSIS_PERIOD}")
print(f"Rate limit delay: {RATE_LIMIT_DELAY} seconds between requests")

# Technical indicators to analyze
TECHNICAL_INDICATORS = {
    'SMA_20': {'function': 'SMA', 'interval': 'daily', 'time_period': 20, 'series_type': 'close'},
    'SMA_50': {'function': 'SMA', 'interval': 'daily', 'time_period': 50, 'series_type': 'close'},
    'EMA_12': {'function': 'EMA', 'interval': 'daily', 'time_period': 12, 'series_type': 'close'},
    'RSI': {'function': 'RSI', 'interval': 'daily', 'time_period': 14, 'series_type': 'close'},
    'MACD': {'function': 'MACD', 'interval': 'daily', 'series_type': 'close'},
    'BBANDS': {'function': 'BBANDS', 'interval': 'daily', 'time_period': 20, 'series_type': 'close'}
}

print(f"Configured {len(TECHNICAL_INDICATORS)} technical indicators for analysis")

Alpha Vantage API key loaded successfully!
Configured for ticker: AAPL
Analysis period: 3month
Rate limit delay: 12 seconds between requests
Configured 6 technical indicators for analysis


## Alpha Vantage API Client Class

Professional client for accessing Alpha Vantage stock market data with comprehensive error handling and analysis capabilities.


In [19]:
class AlphaVantageClient:
    """
    Professional client for Alpha Vantage API with comprehensive stock market data processing.
    
    Provides methods for retrieving stock prices, technical indicators, company fundamentals,
    and generating investment-relevant insights from market data.
    """
    
    def __init__(self, api_key: str, base_url: str = ALPHA_VANTAGE_BASE_URL):
        """Initialize Alpha Vantage client with API credentials."""
        self.api_key = api_key
        self.base_url = base_url
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Investment-Research-Platform/1.0'
        })
    
    def get_stock_quote(self, symbol: str) -> Dict[str, Any]:
        """
        Get real-time stock quote data.
        
        Args:
            symbol: Stock ticker symbol (e.g., 'AAPL')
            
        Returns:
            Dictionary containing current stock price and metadata
        """
        try:
            params = {
                'function': 'GLOBAL_QUOTE',
                'symbol': symbol,
                'apikey': self.api_key
            }
            
            time.sleep(RATE_LIMIT_DELAY)
            response = self.session.get(self.base_url, params=params, timeout=REQUEST_TIMEOUT)
            response.raise_for_status()
            
            data = response.json()
            
            if 'Global Quote' in data:
                quote = data['Global Quote']
                return {
                    'symbol': quote.get('01. Symbol', symbol),
                    'price': float(quote.get('05. Price', 0)),
                    'change': float(quote.get('09. Change', 0)),
                    'change_percent': quote.get('10. Change Percent', '0%').strip('%'),
                    'volume': int(quote.get('06. Volume', 0)) if quote.get('06. Volume', '0').isdigit() else 0,
                    'latest_trading_day': quote.get('07. Latest Trading Day', ''),
                    'previous_close': float(quote.get('08. Previous Close', 0)),
                    'open': float(quote.get('02. Open', 0)),
                    'high': float(quote.get('03. High', 0)),
                    'low': float(quote.get('04. Low', 0)),
                    'timestamp': datetime.now().isoformat()
                }
            
            elif 'Error Message' in data:
                return {'error': data['Error Message']}
            elif 'Note' in data:
                return {'error': f"API limit reached: {data['Note']}"}
            else:
                return {'error': 'Unexpected response format'}
                
        except requests.exceptions.RequestException as e:
            return {'error': f'Network error: {e}'}
        except Exception as e:
            return {'error': f'Processing error: {e}'}
    
    def get_daily_data(self, symbol: str, outputsize: str = 'compact') -> Dict[str, Any]:
        """
        Get daily stock price data.
        
        Args:
            symbol: Stock ticker symbol
            outputsize: 'compact' (100 days) or 'full' (20+ years)
            
        Returns:
            Dictionary containing daily price data
        """
        try:
            params = {
                'function': 'TIME_SERIES_DAILY',
                'symbol': symbol,
                'outputsize': outputsize,
                'apikey': self.api_key
            }
            
            time.sleep(RATE_LIMIT_DELAY)
            response = self.session.get(self.base_url, params=params, timeout=REQUEST_TIMEOUT)
            response.raise_for_status()
            
            data = response.json()
            
            if 'Time Series (Daily)' in data:
                time_series = data['Time Series (Daily)']
                processed_data = []
                
                for date_str, values in time_series.items():
                    processed_data.append({
                        'date': date_str,
                        'open': float(values['1. open']),
                        'high': float(values['2. high']),
                        'low': float(values['3. low']),
                        'close': float(values['4. close']),
                        'volume': int(values['5. volume'])
                    })
                
                # Sort by date (newest first)
                processed_data.sort(key=lambda x: x['date'], reverse=True)
                
                return {
                    'symbol': symbol,
                    'data': processed_data,
                    'count': len(processed_data),
                    'latest_date': processed_data[0]['date'] if processed_data else None,
                    'latest_close': processed_data[0]['close'] if processed_data else None
                }
            
            elif 'Error Message' in data:
                return {'error': data['Error Message']}
            elif 'Note' in data:
                return {'error': f"API limit reached: {data['Note']}"}
            else:
                return {'error': 'Unexpected response format'}
                
        except Exception as e:
            return {'error': f'Error retrieving daily data: {e}'}
    
    def get_technical_indicator(self, symbol: str, indicator_config: Dict[str, Any]) -> Dict[str, Any]:
        """
        Get technical indicator data for a stock.
        
        Args:
            symbol: Stock ticker symbol
            indicator_config: Configuration for the technical indicator
            
        Returns:
            Dictionary containing technical indicator data
        """
        try:
            params = {
                'function': indicator_config['function'],
                'symbol': symbol,
                'interval': indicator_config['interval'],
                'apikey': self.api_key
            }
            
            # Add optional parameters
            if 'time_period' in indicator_config:
                params['time_period'] = indicator_config['time_period']
            if 'series_type' in indicator_config:
                params['series_type'] = indicator_config['series_type']
            
            time.sleep(RATE_LIMIT_DELAY)
            response = self.session.get(self.base_url, params=params, timeout=REQUEST_TIMEOUT)
            response.raise_for_status()
            
            data = response.json()
            
            # Find the technical analysis key (varies by indicator)
            tech_key = None
            for key in data.keys():
                if 'Technical Analysis' in key or key.startswith('Technical'):
                    tech_key = key
                    break
            
            if tech_key and tech_key in data:
                tech_data = data[tech_key]
                processed_data = []
                
                for date_str, values in tech_data.items():
                    entry = {'date': date_str}
                    for value_key, value in values.items():
                        # Clean up the key name
                        clean_key = value_key.split('. ')[-1] if '. ' in value_key else value_key
                        try:
                            entry[clean_key] = float(value)
                        except (ValueError, TypeError):
                            entry[clean_key] = value
                    processed_data.append(entry)
                
                # Sort by date (newest first)
                processed_data.sort(key=lambda x: x['date'], reverse=True)
                
                return {
                    'symbol': symbol,
                    'indicator': indicator_config['function'],
                    'data': processed_data[:50],  # Latest 50 values
                    'count': len(processed_data),
                    'latest_date': processed_data[0]['date'] if processed_data else None
                }
            
            elif 'Error Message' in data:
                return {'error': data['Error Message']}
            elif 'Note' in data:
                return {'error': f"API limit reached: {data['Note']}"}
            else:
                return {'error': f'No technical data found. Available keys: {list(data.keys())}'}
                
        except Exception as e:
            return {'error': f'Error retrieving {indicator_config["function"]}: {e}'}
    
    def calculate_basic_metrics(self, daily_data: List[Dict]) -> Dict[str, float]:
        """Calculate basic financial metrics from daily price data."""
        if not daily_data or len(daily_data) < 2:
            return {'error': 'Insufficient data for metrics calculation'}
        
        try:
            # Sort by date (oldest first for calculations)
            sorted_data = sorted(daily_data, key=lambda x: x['date'])
            
            current_price = sorted_data[-1]['close']
            previous_price = sorted_data[-2]['close'] if len(sorted_data) > 1 else current_price
            
            # Price changes
            daily_change = current_price - previous_price
            daily_change_pct = (daily_change / previous_price * 100) if previous_price != 0 else 0
            
            # Volatility (standard deviation of daily returns)
            if len(sorted_data) >= 20:
                returns = []
                for i in range(1, min(21, len(sorted_data))):  # Last 20 days
                    prev_close = sorted_data[-(i+1)]['close']
                    curr_close = sorted_data[-i]['close']
                    if prev_close != 0:
                        returns.append((curr_close - prev_close) / prev_close)
                
                if returns:
                    mean_return = sum(returns) / len(returns)
                    variance = sum((r - mean_return) ** 2 for r in returns) / len(returns)
                    volatility = math.sqrt(variance) * 100  # As percentage
                else:
                    volatility = 0
            else:
                volatility = 0
            
            # Trading volume analysis
            recent_volumes = [d['volume'] for d in sorted_data[-10:]]  # Last 10 days
            avg_volume = sum(recent_volumes) / len(recent_volumes) if recent_volumes else 0
            
            # Price range analysis
            recent_highs = [d['high'] for d in sorted_data[-20:]]  # Last 20 days
            recent_lows = [d['low'] for d in sorted_data[-20:]]  # Last 20 days
            
            range_high = max(recent_highs) if recent_highs else current_price
            range_low = min(recent_lows) if recent_lows else current_price
            
            # Position within range
            if range_high != range_low:
                position_in_range = (current_price - range_low) / (range_high - range_low) * 100
            else:
                position_in_range = 50  # Middle if no range
            
            return {
                'current_price': current_price,
                'daily_change': daily_change,
                'daily_change_pct': daily_change_pct,
                'volatility_20d': volatility,
                'avg_volume_10d': avg_volume,
                'range_high_20d': range_high,
                'range_low_20d': range_low,
                'position_in_range': position_in_range,
                'data_points': len(sorted_data)
            }
            
        except Exception as e:
            return {'error': f'Metrics calculation error: {e}'}

# Initialize Alpha Vantage client
if ALPHA_VANTAGE_API_KEY:
    av_client = AlphaVantageClient(ALPHA_VANTAGE_API_KEY)
    print("Alpha Vantage client initialized successfully!")
else:
    print("Cannot initialize Alpha Vantage client - API key required")
    av_client = None

Alpha Vantage client initialized successfully!


## Stock Data Retrieval

Fetch current stock quote and historical price data for analysis.


In [20]:
if av_client:
    print(f"Retrieving stock data for {TEST_TICKER}...")
    
    # Get current stock quote
    quote_data = av_client.get_stock_quote(TEST_TICKER)
    
    if 'error' not in quote_data:
        print(f"Current Quote for {quote_data['symbol']}:")
        print(f"  Price: ${quote_data['price']:.2f}")
        print(f"  Change: {quote_data['change']:+.2f} ({quote_data['change_percent']}%)")
        print(f"  Volume: {quote_data['volume']:,}")
        print(f"  Latest Trading Day: {quote_data['latest_trading_day']}")
    else:
        print(f"Error fetching quote: {quote_data['error']}")
    
    # Get historical daily data
    daily_data = av_client.get_daily_data(TEST_TICKER, outputsize='compact')
    
    if 'error' not in daily_data:
        print(f"Historical Data: {daily_data['count']} data points, latest: {daily_data['latest_date']}")
    else:
        print(f"Error fetching daily data: {daily_data['error']}")
    
    print("Data retrieval completed")
    
else:
    print("Alpha Vantage client not available - please configure API key")

Retrieving stock data for AAPL...
Current Quote for AAPL:
  Price: $0.00
  Change: +0.00 (0%)
  Volume: 0
  Latest Trading Day: 
Current Quote for AAPL:
  Price: $0.00
  Change: +0.00 (0%)
  Volume: 0
  Latest Trading Day: 
Historical Data: 100 data points, latest: 2025-09-19
Data retrieval completed
Historical Data: 100 data points, latest: 2025-09-19
Data retrieval completed


## Technical Indicators Analysis

Retrieve and analyze key technical indicators for investment decision making.


In [21]:
if av_client and 'daily_data' in locals() and 'error' not in daily_data:
    print(f"Analyzing technical indicators for {TEST_TICKER}...")
    
    # Store technical indicator results
    technical_results = {}
    
    # Retrieve each technical indicator
    for indicator_name, config in TECHNICAL_INDICATORS.items():
        indicator_data = av_client.get_technical_indicator(TEST_TICKER, config)
        technical_results[indicator_name] = indicator_data
        
        if 'error' not in indicator_data:
            # Display latest values and interpretation
            if indicator_data['data']:
                latest = indicator_data['data'][0]
                print(f"{indicator_name}: {indicator_data['latest_date']}")
                
                # Investment interpretation
                if indicator_name == 'RSI':
                    rsi_value = latest.get('RSI', 0)
                    if rsi_value > 70:
                        interpretation = "Overbought - potential sell signal"
                    elif rsi_value < 30:
                        interpretation = "Oversold - potential buy signal"
                    else:
                        interpretation = "Neutral territory"
                    print(f"  RSI: {rsi_value:.2f} - {interpretation}")
                
                elif indicator_name in ['SMA_20', 'SMA_50', 'EMA_12']:
                    ma_value = latest.get(list(latest.keys())[1], 0) if len(latest) > 1 else 0
                    current_price = quote_data.get('price', 0) if 'quote_data' in locals() else 0
                    if current_price > ma_value:
                        interpretation = f"Price above {indicator_name} - bullish signal"
                    elif current_price < ma_value:
                        interpretation = f"Price below {indicator_name} - bearish signal"
                    else:
                        interpretation = f"Price near {indicator_name} - neutral"
                    print(f"  → {interpretation}")
                
                elif indicator_name == 'MACD':
                    macd_value = latest.get('MACD', 0)
                    signal_value = latest.get('MACD_Signal', 0)
                    if macd_value > signal_value:
                        interpretation = "MACD above signal - potential bullish momentum"
                    elif macd_value < signal_value:
                        interpretation = "MACD below signal - potential bearish momentum"
                    else:
                        interpretation = "MACD near signal line - watch for crossover"
                    print(f"  → {interpretation}")
                
                elif indicator_name == 'BBANDS':
                    upper_band = latest.get('Real Upper Band', 0)
                    lower_band = latest.get('Real Lower Band', 0)
                    current_price = quote_data.get('price', 0) if 'quote_data' in locals() else 0
                    
                    if current_price > upper_band:
                        interpretation = "Price above upper band - potential overbought"
                    elif current_price < lower_band:
                        interpretation = "Price below lower band - potential oversold"
                    else:
                        interpretation = "Price within bands - normal range"
                    print(f"  → {interpretation}")
        
        else:
            print(f"{indicator_name}: Error - {indicator_data['error']}")
        
        # Rate limiting between requests
        if indicator_name != list(TECHNICAL_INDICATORS.keys())[-1]:  # Not the last one
            time.sleep(RATE_LIMIT_DELAY)
    
    print("Technical analysis completed")
    
else:
    print("Stock data not available for technical analysis")

Analyzing technical indicators for AAPL...
SMA_20: 2025-09-19
  → Price below SMA_20 - bearish signal
SMA_20: 2025-09-19
  → Price below SMA_20 - bearish signal
SMA_50: 2025-09-19
  → Price below SMA_50 - bearish signal
SMA_50: 2025-09-19
  → Price below SMA_50 - bearish signal
EMA_12: 2025-09-19
  → Price below EMA_12 - bearish signal
EMA_12: 2025-09-19
  → Price below EMA_12 - bearish signal
RSI: 2025-09-19
  RSI: 67.93 - Neutral territory
RSI: 2025-09-19
  RSI: 67.93 - Neutral territory
MACD: Error - No technical data found. Available keys: ['Information']
MACD: Error - No technical data found. Available keys: ['Information']
BBANDS: 2025-09-19
  → Price below lower band - potential oversold
Technical analysis completed
BBANDS: 2025-09-19
  → Price below lower band - potential oversold
Technical analysis completed


## LangChain Analysis and Investment Insights

Use LangChain to process and analyze the collected financial data for intelligent investment insights.


In [22]:
if av_client and 'daily_data' in locals() and 'error' not in daily_data:
    print(f"Generating investment insights for {TEST_TICKER}...")
    print("=" * 50)
    
    # Calculate basic financial metrics
    print("Calculating financial metrics...")
    metrics = av_client.calculate_basic_metrics(daily_data['data'])
    
    if 'error' not in metrics:
        print(f"\nFinancial Metrics Analysis:")
        print(f"  Current Price: ${metrics['current_price']:.2f}")
        print(f"  Daily Change: {metrics['daily_change']:+.2f} ({metrics['daily_change_pct']:+.1f}%)")
        print(f"  20-Day Volatility: {metrics['volatility_20d']:.2f}%")
        print(f"  Average Volume (10d): {metrics['avg_volume_10d']:,.0f}")
        print(f"  20-Day Range: ${metrics['range_low_20d']:.2f} - ${metrics['range_high_20d']:.2f}")
        print(f"  Position in Range: {metrics['position_in_range']:.1f}%")
        print(f"  Data Points: {metrics['data_points']}")
    else:
        print(f"Error calculating metrics: {metrics['error']}")
    
    # Create comprehensive analysis document using LangChain
    print("\nCreating comprehensive analysis document...")
    
    # Compile all analysis data into a document
    analysis_text = f"""
STOCK ANALYSIS REPORT FOR {TEST_TICKER}
Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

CURRENT MARKET DATA:
"""
    
    if 'quote_data' in locals() and 'error' not in quote_data:
        analysis_text += f"""
Current Price: ${quote_data['price']:.2f}
Daily Change: {quote_data['change']:+.2f} ({quote_data['change_percent']}%)
Trading Volume: {quote_data['volume']:,}
Price Range: ${quote_data['low']:.2f} - ${quote_data['high']:.2f}
Previous Close: ${quote_data['previous_close']:.2f}
"""
    
    if 'metrics' in locals() and 'error' not in metrics:
        analysis_text += f"""

FINANCIAL METRICS:
20-Day Volatility: {metrics['volatility_20d']:.2f}%
Average Volume (10 days): {metrics['avg_volume_10d']:,.0f}
20-Day Price Range: ${metrics['range_low_20d']:.2f} - ${metrics['range_high_20d']:.2f}
Current Position in Range: {metrics['position_in_range']:.1f}%
"""
    
    # Add technical indicators summary
    if 'technical_results' in locals():
        analysis_text += "\n\nTECHNICAL INDICATORS SUMMARY:\n"
        
        for indicator_name, data in technical_results.items():
            if 'error' not in data and data.get('data'):
                latest = data['data'][0]
                analysis_text += f"\n{indicator_name}:\n"
                for key, value in latest.items():
                    if key != 'date' and isinstance(value, (int, float)):
                        analysis_text += f"  {key}: {value:.4f}\n"
    
    # Create LangChain document
    doc = Document(page_content=analysis_text)
    
    # Split document for processing
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    
    chunks = text_splitter.split_documents([doc])
    
    print(f"Analysis document created with {len(chunks)} chunks")
    print(f"Total analysis length: {len(analysis_text)} characters")
    
    # Investment decision framework
    print("\nINVESTMENT DECISION FRAMEWORK:")
    print("=" * 35)
    
    bullish_signals = []
    bearish_signals = []
    neutral_signals = []
    
    # Analyze price momentum
    if 'metrics' in locals() and 'error' not in metrics:
        if metrics['daily_change_pct'] > 2:
            bullish_signals.append(f"Strong daily gain of {metrics['daily_change_pct']:.1f}%")
        elif metrics['daily_change_pct'] < -2:
            bearish_signals.append(f"Significant daily loss of {metrics['daily_change_pct']:.1f}%")
        else:
            neutral_signals.append(f"Moderate daily change of {metrics['daily_change_pct']:.1f}%")
        
        # Position in trading range
        if metrics['position_in_range'] > 80:
            bearish_signals.append("Price near 20-day high - potential resistance")
        elif metrics['position_in_range'] < 20:
            bullish_signals.append("Price near 20-day low - potential support")
        else:
            neutral_signals.append("Price in middle of recent range")
        
        # Volatility analysis
        if metrics['volatility_20d'] > 30:
            bearish_signals.append(f"High volatility of {metrics['volatility_20d']:.1f}% indicates uncertainty")
        elif metrics['volatility_20d'] < 15:
            bullish_signals.append(f"Low volatility of {metrics['volatility_20d']:.1f}% suggests stability")
        else:
            neutral_signals.append(f"Moderate volatility of {metrics['volatility_20d']:.1f}%")
    
    # Analyze technical indicators
    if 'technical_results' in locals():
        # RSI analysis
        if 'RSI' in technical_results and 'error' not in technical_results['RSI']:
            rsi_data = technical_results['RSI'].get('data', [])
            if rsi_data:
                rsi_value = rsi_data[0].get('RSI', 50)
                if rsi_value > 70:
                    bearish_signals.append(f"RSI overbought at {rsi_value:.1f}")
                elif rsi_value < 30:
                    bullish_signals.append(f"RSI oversold at {rsi_value:.1f}")
                else:
                    neutral_signals.append(f"RSI neutral at {rsi_value:.1f}")
        
        # Moving average analysis
        current_price = quote_data.get('price', 0) if 'quote_data' in locals() else 0
        
        if 'SMA_20' in technical_results and current_price > 0:
            sma20_data = technical_results['SMA_20'].get('data', [])
            if sma20_data:
                sma20_value = list(sma20_data[0].values())[1] if len(sma20_data[0]) > 1 else 0
                if current_price > sma20_value * 1.02:  # 2% above
                    bullish_signals.append("Price well above 20-day SMA")
                elif current_price < sma20_value * 0.98:  # 2% below
                    bearish_signals.append("Price well below 20-day SMA")
                else:
                    neutral_signals.append("Price near 20-day SMA")
    
    # Display signals
    print(f"\nBULLISH SIGNALS ({len(bullish_signals)}):")
    for signal in bullish_signals:
        print(f"  + {signal}")
    
    print(f"\nBEARISH SIGNALS ({len(bearish_signals)}):")
    for signal in bearish_signals:
        print(f"  - {signal}")
    
    print(f"\nNEUTRAL FACTORS ({len(neutral_signals)}):")
    for signal in neutral_signals:
        print(f"  = {signal}")
    
    # Overall recommendation
    print(f"\nOVERALL INVESTMENT ASSESSMENT:")
    print("-" * 35)
    
    bullish_score = len(bullish_signals)
    bearish_score = len(bearish_signals)
    
    if bullish_score > bearish_score + 1:
        recommendation = "BULLISH - Consider buying opportunities"
        confidence = min(90, 60 + (bullish_score - bearish_score) * 10)
    elif bearish_score > bullish_score + 1:
        recommendation = "BEARISH - Exercise caution or consider selling"
        confidence = min(90, 60 + (bearish_score - bullish_score) * 10)
    else:
        recommendation = "NEUTRAL - Hold current positions, monitor closely"
        confidence = 50 + abs(bullish_score - bearish_score) * 5
    
    print(f"Recommendation: {recommendation}")
    print(f"Confidence Level: {confidence}%")
    
    print(f"\nAnalysis completed at {datetime.now().strftime('%H:%M:%S')}")
    
else:
    print("Stock data not available for investment analysis")

Generating investment insights for AAPL...
Calculating financial metrics...

Financial Metrics Analysis:
  Current Price: $245.50
  Daily Change: +7.62 (+3.2%)
  20-Day Volatility: 1.52%
  Average Volume (10d): 66,540,655
  20-Day Range: $224.69 - $246.30
  Position in Range: 96.3%
  Data Points: 100

Creating comprehensive analysis document...
Analysis document created with 1 chunks
Total analysis length: 577 characters

INVESTMENT DECISION FRAMEWORK:

BULLISH SIGNALS (2):
  + Strong daily gain of 3.2%
  + Low volatility of 1.5% suggests stability

BEARISH SIGNALS (1):
  - Price near 20-day high - potential resistance

NEUTRAL FACTORS (1):
  = RSI neutral at 67.9

OVERALL INVESTMENT ASSESSMENT:
-----------------------------------
Recommendation: NEUTRAL - Hold current positions, monitor closely
Confidence Level: 55%

Analysis completed at 23:00:31


## Data Export and Portfolio Integration

Export analysis results and prepare data for portfolio management integration.


In [23]:
if 'quote_data' in locals() and 'daily_data' in locals():
    print("Exporting analysis results...")
    
    # Create data directory if it doesn't exist
    import os
    data_dir = "data"
    os.makedirs(data_dir, exist_ok=True)
    
    # Create comprehensive export dataset
    export_data = {
        'analysis_timestamp': datetime.now().isoformat(),
        'ticker': TEST_TICKER,
        'current_quote': quote_data if 'error' not in quote_data else None,
        'daily_data_summary': {
            'count': daily_data.get('count', 0),
            'latest_date': daily_data.get('latest_date', ''),
            'latest_close': daily_data.get('latest_close', 0)
        } if 'error' not in daily_data else None,
        'technical_indicators': {}
    }
    
    # Add technical indicator results
    if 'technical_results' in locals():
        for indicator_name, data in technical_results.items():
            if 'error' not in data:
                export_data['technical_indicators'][indicator_name] = {
                    'count': data.get('count', 0),
                    'latest_values': data['data'][0] if data.get('data') else None,
                    'latest_date': data.get('latest_date', '')
                }
            else:
                export_data['technical_indicators'][indicator_name] = {'error': data['error']}
    
    # Add investment signals
    if 'bullish_signals' in locals():
        export_data['investment_analysis'] = {
            'bullish_signals': bullish_signals,
            'bearish_signals': bearish_signals,
            'neutral_signals': neutral_signals,
            'recommendation': recommendation if 'recommendation' in locals() else 'No recommendation generated',
            'confidence': confidence if 'confidence' in locals() else 0
        }
    
    # Add recent price history (last 20 days)
    if daily_data.get('data'):
        export_data['price_history'] = daily_data['data'][:20]  # Last 20 trading days
    
    # Export to JSON file
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    json_filename = os.path.join(data_dir, f"alphavantage_analysis_{TEST_TICKER}_{timestamp}.json")
    
    try:
        with open(json_filename, 'w') as f:
            json.dump(export_data, f, indent=2, default=str)
        print(f"Analysis exported to: {json_filename}")
    except Exception as e:
        print(f"Error exporting JSON: {e}")
    
    # Create DataFrame for CSV export
    try:
        # Prepare daily data for CSV
        if daily_data.get('data'):
            df_data = []
            for day in daily_data['data'][:60]:  # Last 60 trading days
                row = {
                    'date': day['date'],
                    'open': day['open'],
                    'high': day['high'],
                    'low': day['low'],
                    'close': day['close'],
                    'volume': day['volume']
                }
                
                # Add current metrics if available
                if 'quote_data' in locals() and day['date'] == quote_data.get('latest_trading_day'):
                    row['current_price'] = quote_data.get('price', day['close'])
                    row['daily_change'] = quote_data.get('change', 0)
                    row['daily_change_pct'] = quote_data.get('change_percent', '0%')
                
                df_data.append(row)
            
            df = pd.DataFrame(df_data)
            csv_filename = os.path.join(data_dir, f"alphavantage_prices_{TEST_TICKER}_{timestamp}.csv")
            df.to_csv(csv_filename, index=False)
            print(f"Price data exported to: {csv_filename}")
            print(f"Export summary: {len(df)} records, {df['date'].min()} to {df['date'].max()}")
            
    except Exception as e:
        print(f"Error creating CSV export: {e}")
    
    # Portfolio integration summary
    if 'recommendation' in locals():
        print(f"Investment recommendation: {recommendation}")
        if 'BULLISH' in recommendation:
            print("  Portfolio action: Consider increasing position")
        elif 'BEARISH' in recommendation:
            print("  Portfolio action: Consider reducing position or hedging")
        else:
            print("  Portfolio action: Maintain current allocation")
    
    print("Analysis export completed")
    
else:
    print("No data available for export")

Exporting analysis results...
Analysis exported to: data\alphavantage_analysis_AAPL_20250920_230031.json
Price data exported to: data\alphavantage_prices_AAPL_20250920_230031.csv
Export summary: 60 records, 2025-06-26 to 2025-09-19
Investment recommendation: NEUTRAL - Hold current positions, monitor closely
  Portfolio action: Maintain current allocation
Analysis export completed
